# Materials

## Concrete

## Rebars

### Bars with distinct yield point

### Cold worked deformed bars



In [1]:
ecy = 0.002
ecu = 0.0035
gammac = 1.5
gammas = 1.15
Es = 2e5

function calc_fc(ec, fck)
    if ec < 0.0 || ec > ecu
        return 0.0
    elseif ec >= ecy
        return fck / gammac^2
    else
        e = ec / ecy
        return (2*e - e*e) * fck / gammac^2
    end
end

function calc_fc(ec::Vector, fck::Number)
    for e in ec
        println(e, ' ', calc_fc(e, fck))
    end
end

ec = [-0.01, 0.0, 0.0005, 0.001, 0.0015, 0.002, 0.0025, 0.003, 0.0035, 0.004]
for e in ec
    println(e, " ", calc_fc(e, 25))
end
println("\n")
calc_fc(ec, 25)

-0.01 0.0
0.0 0.0
0.0005 4.861111111111111
0.001 8.333333333333334
0.0015 10.416666666666666
0.002 11.11111111111111
0.0025 11.11111111111111
0.003 11.11111111111111
0.0035 11.11111111111111
0.004 0.0


-0.01 0.0
0.0 0.0
0.0005 4.861111111111111
0.001 8.333333333333334
0.0015 10.416666666666666
0.002 11.11111111111111
0.0025 11.11111111111111
0.003 11.11111111111111
0.0035 11.11111111111111
0.004 0.0


In [3]:
function calc_xumax(fy)
    return ecy / (ecy + ecu + fy / (gammas * Es))
end

function calc_xumax(fy::Vector)
    for i = 1:length(fy)
        println(fy[i], " ", calc_xumax(fy[i]))
    end
end

println(calc_xumax(250))
println(calc_xumax(415))
println(calc_xumax(500))

fy = [250, 415, 500]
calc_xumax(fy)

0.30363036303630364
0.2738095238095238
0.26062322946175637
250 0.30363036303630364
415 0.2738095238095238
500 0.26062322946175637


.


In [31]:
function calc_fs(es, fy, CWDB=true)
    es = abs(es)
    if CWDB
        s = [0.8, 0.85, 0.9, 0.95, 0.975, 1.0]
        rs = [0.0, 0.0001, 0.0003, 0.0007, 0.001, 0.002]
        e = s * fy / (gammas * Es) + rs
        if es <= e[1]
            return es * Es
        elseif es >= e[6]
            fy / gammas
        else
            i = 1
            while es > e[i+1]
                i += 1
            end
            x1 = e[i]
            y1 = s[i] * fy / gammas
            x2 = e[i+1]
            y2 = s[i+1] * fy / gammas
            return y1 + (y2 - y1) / (x2 - x1) * (es - x1)
        end
    else
        esy = fy / (gammas * Es)
        if es <= esy
            return es * Es
        else
            return fy / (gammas)
        end
    end
end

function calc_fs(es::Vector, fy::Number, CWDB::Bool=true)
    fs = zeros(es)
    for i = 1:length(es)
        fs[i] = calc_fs(es[i], fy, CWDB)
    end
    return fs
end

#calc_fs(250/(1.15*2e5), 250, false)
es = 0.0015
println(es, " ", calc_fs(es, 415))
es = 1*500/(1.15*2e5) + 0.002
println(es, " ", calc_fs(es, 500))

fy = 415
es = [0, 0.5, 0.8, 0.82, 0.85, 0.88, 0.9, 0.92, 0.95, 0.96, 0.975, 0.98, 1.0, 1.0] * fy / (gammas * Es)
es += [0, 0, 0, 0.0001, 0.0001, 0.0003, 0.0003, 0.0007, 0.0007, 0.001, 0.001, 0.002, 0.002, 0.0025]
fs = calc_fs(es, fy, true)
println(es, "\n", fs)

0.0015 294.0571428571429
0.004173913043478261 434.7826086956522
[0.0,0.000902174,0.00144348,0.00157957,0.0016337,0.00188783,0.00192391,0.00236,0.00241413,0.00273217,0.00275924,0.00376826,0.00380435,0.00430435]
[0.0,180.435,288.696,301.604,306.739,322.539,324.783,340.834,342.826,351.14,351.848,360.558,360.87,360.87]


In [15]:
function calc_k1(fy::Number)
    return ecu / (ecy + ecu + fy / (gammas * Es))
end

function calc_k1(fy::Array{Number, 1})
    x = zeros(fy)
    for i = 1:length(fy)
        x[i] = calc_k1(fy[i])
    end
    return x
end

println(calc_k1(Float64[250, 415, 500]))

[0.531353,0.479167,0.456091]


In [4]:
function calc_k2()
    return 68 // 189
end

println(calc_k2())

68//189


In [5]:
function calc_k3()
    return 99 // 238
end

println(calc_k3())

99//238


In [20]:
function calc_Q(fy::Number, xubyd::Number)
    k2 = calc_k2()
    k3 = calc_k3()
    return k2 * xubyd * (1.0 - k3 * xubyd)
end

function calc_Mulim(fck::Number, fy::Number, b::Number, d::Number)
    xumaxbyd = calc_k1(fy)
    Q = calc_Q(fy, xumaxbyd)
    return Q * fck * b * d^2
end

fy = 500.0
xumaxbyd = calc_k1(fy)
println(calc_Q(fy, xumaxbyd))
println(calc_Mulim(20.0, 415.0, 230.0, 415.0)/1e6)

0.13296405972697412
109.35741025171642


In [26]:
function rect_design(Mu, Vu, Tu, fck, fy, b, d)
    Mu = abs(Mu)
    Vu = abs(Vu)
    if Mu <= calc_Mulim(fck, fy, b, d)
        println("Singly reinforced beam. Mu = ", Mu, " < ", calc_Mulim(fck, fy, b, d))
        return true
    else
        println("Doubly reinforced beam")
        return false
    end
end

println(rect_design(110e6, 90e3, 0.0, 20.0, 415.0, 230.0, 415.0))

Doubly reinforced beam
false
